# ベイズ最適化（hyperopt）でのパラメータ探索

## 必要なライブラリのインポート

In [14]:
import numpy as np
import pandas as pd

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

## データセットの取得

データセットはこちらのURLにアップデートされているものを使用\
https://github.com/ghmagazine/kagglebook/tree/master/input/sample-data

In [5]:
# train_xは学習データ、train_yは目的変数、test_xはテストデータ
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [6]:
train.head()

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,1,1,1,0,2016,9,17,24201,1


In [7]:
train.shape

(10000, 29)

In [8]:
train_x = train.iloc[:, :28]
train_y = train['target']
test_x = test.copy()

In [11]:
# KFoldクロスバリデーションによる分割の1つを使用し、学習データとバリデーションデータに分ける
kf = KFold(n_splits=4, shuffle=True, random_state=71)
##  トレーニングデータセットとテストデータセットにそれぞれ振り分けた要素番号のリスト
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

## hyperoptライブラリの使用

In [12]:
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}

hyperoptを使用したパラメータ探索は、以下のようになる。
1. チューニングしたいパラメータを引数にとり、最小化したい評価指標のスコアを返す関数を作成する。
  - その関数では、モデルを引数のパラメータで学習させ、バリデーションデータへの予測を行い、評価指標のスコアを計算する処理を行う。
2. hyperoptのfmin関数に、その作成した関数、探索するパラメータの空間、探索回数などを指定することで探索する。

In [17]:
def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする
    
    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])
    
    # Modelクラスを定義しているものとする
    ## Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'pramas: {params}, logloss: {score:.4f}')
    
    # 情報を記録しておく
    history.append((params, score))
    
    return {'loss': score, 'status': STATUS_OK}



In [ ]:
# 探索するパラメータの空間を指定する
space2 = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1)
}

# hyperoptによるパラメータ探索の実行
max_evals = 10
trials = Trials()
history = []
fmin(score, space2, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# (trialsからも情報を取得できるが、パラメータの取得がやや行いづらいため)
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params: {best[0]}, score: {best[1]:.4f}')